In [ ]:
import socket
import threading
import time
import cv2, os, json, pickle, uuid, sys

In [ ]:

import numpy as np
from NeuralNetStructure import accuracy, activation, dropout,  layer, loss, nnet, optimizers, pool
from preprocessor import Preprocessor
from NeuralNetStructure.imagetools import ImageTools

In [ ]:
PORT = 5050
SERVER = socket.gethostbyname(socket.gethostname())
ADDR = (SERVER, PORT)
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)
HEADER = 2
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = "!DISCONNECT"

In [ ]:
SERVER

In [ ]:
def getCmd():
    global command
    cap = cv2.VideoCapture(0)
    CLASSES = ["ARM_CROSS", "BACK_HAND", "FIST", "FRONT_HAND", "POINT_LEFT", "POINT_RIGHT", "PRAN_MUDRA"]
    mordecAI = nnet.Model.load("../VisualCortex/handsigns_classifier.nucleus")

    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (20,20)
    fontScale              = 0.5
    fontColor              = (72,251,0)
    thickness              = 1
    lineType               = 2

    while True:
        ret, frame = cap.read()

        frame = frame[100:300, 220:420, :]
        img = cv2.resize(frame, (100, 100))
        img = ImageTools.channelLastToFirst(img)
        output = mordecAI.forward(np.expand_dims(img, 0), False)
        prediction = CLASSES[np.argmax(output)] if np.max(output) > 0.9 else "None"
        cv2.putText(frame,f'{prediction}', 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        thickness,
        lineType)

        command = prediction

        cv2.imshow("test", frame)
        if cv2.waitKey(1) == 27:
            cv2.destroyAllWindows()
            break

In [ ]:
mSight = threading.Thread(target=getCmd)

In [ ]:
def handleClient(conn, addr, server=None):
    
    global command
    MASTER = True if server else False
    print(f"[NEW CONNECTION] : {addr} connected.")

    mSight.start()
    connected = True
    while connected:

        try:
            if command != "None":
                conn.send(f"{command}".encode(FORMAT))
                print(f"mordecAI: {command}")
        except Exception as e:
            print("Engine shut down.")
            break

        #command = "None"
      
    conn.close()
    if server is not None:
        server.close()

def start():
    server.listen()
    print(f"[LISTENING]: Server is listening on {SERVER}")
    conn, addr = server.accept()
    master = threading.Thread(target=handleClient, args=[conn, addr, server])
    master.start()
    while True:
        try:
            conn, addr = server.accept() # this will throw an exception when server closes. 
            client = threading.Thread(target=handleClient, args=[conn, addr])
            client.start()
        except Exception as e:
            print("mordecAI shut down.")
            break



In [ ]:
command = "None"
print(f"[STARTING] Server is starting...")
start()


In [1]:
%pip install numba

     ---------------------------------------- 2.4/2.4 MB 18.8 MB/s eta 0:00:00
     --------------------------------------- 23.2/23.2 MB 32.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\nurei\source\repos\MARKAM\MSight\Genesis\venvMSight\Scripts\python.exe -m pip install --upgrade pip' command.
